In [1]:
import datetime
from langchain.agents import AgentExecutor
from langchain.agents.structured_chat.base import StructuredChatAgent
from langchain.prompts import PromptTemplate
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL
from mdcrow.utils import _make_llm

In [2]:
prompt = "Write me a code to run simulation of fibronectin."
model = "gpt-4o-2024-08-06"

In [3]:
# Parameters
prompt = "Download and clean protein 1A3N."


In [4]:
llm = _make_llm(model, temp=0.1, streaming=True)

python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)
tools = [repl_tool]

structured_prompt = PromptTemplate(
    input_variables=["input"],
    template="""
    You are an expert molecular dynamics scientist, and your 
    task is to respond to the question or 
    solve the problem in its entirety to the best of your ability. 
    If any part of the task requires you to perform an action that 
    you are not capable of completing, please write a runnable 
    Python script for that step and run it. For literature papers, 
    use and process papers from the `paper_collection` folder. 
    For .pdb files, download them from the RSCB website using `requests`. 
    To preprocess PDB files, you will use PDBFixer. 
    To get information about proteins, retrieve data from the UniProt database. 
    For anything related to simulations, you will use OpenMM, 
    and for anything related to analyses, you will use MDTraj. 

    You can only respond with a single complete
    'Thought, Action, Action Input' format
    OR a single 'Final Answer' format.

    Complete format:
    Thought: (reflect on your progress and decide what to do next)
    Action:
    ```
    {{
        "action": (the action name, it should be the name of a tool),
        "action_input": (the input string for the action)
    }}
    '''

    OR

    Final Answer: (the final response to the original input
    question, once all steps are complete)

    You are required to use the tools provided,
    using the most specific tool
    available for each action.
    Your final answer should contain all information
    necessary to answer the question and its subquestions.
    Before you finish, reflect on your progress and make
    sure you have addressed the question in its entirety.

    Here is the input:
    Question: {input} """,
)

agent = StructuredChatAgent.from_llm_and_tools(
    llm=llm, tools=tools
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [5]:
now = datetime.datetime.now()
date = now.strftime("%Y-%m-%d")
print("date:",date)
time = now.strftime("%H:%M:%S")
print("time:",time)
print("LLM: ",llm.model_name,"\nTemperature: ",llm.temperature)

date: 2024-10-17
time: 23:50:22
LLM:  gpt-4o-2024-08-06 
Temperature:  0.1


In [6]:
messages = structured_prompt.format(input=prompt)
agent_executor.invoke({"input": messages})



> Entering new AgentExecutor chain...


Thought

:

 To

 download

...

 this

 script

.

The environment where the code is being executed does not have the `requests` library available, and I am unable to install it due to restrictions. Therefore, I cannot proceed with downloading and cleaning the PDB file for protein 1A3N in this environment. However, I can provide you with a Python script that you can run in your local environment to achieve this task.

Final Answer:
```python
import requests
from pdbfixer import PDBFixer
from openmm.app import PDBFile

# Download the PDB file for protein 1A3N
def download_pdb(pdb_id):
    url = f'https://files.rcsb.org/download/{pdb_id}.pdb'
    response = requests.get(url)
    if response.status_code == 200:
        with open(f'{pdb_id}.pdb', 'w') as file:
            file.write(response.text)
    else:
        raise Exception('Failed to download PDB file')

# Clean the PDB file using PDBFixer
def clean_pdb(pdb_id):
    fixer = PDBFixer(filename=f'{pdb_id}.pdb')
    fixer.findMissingResidues()
    fixer.findNonstandardResidues()
    fi

{'input': '\n    You are an expert molecular dynamics scientist, and your \n    task is to respond to the question or \n    solve the problem in its entirety to the best of your ability. \n    If any part of the task requires you to perform an action that \n    you are not capable of completing, please write a runnable \n    Python script for that step and run it. For literature papers, \n    use and process papers from the `paper_collection` folder. \n    For .pdb files, download them from the RSCB website using `requests`. \n    To preprocess PDB files, you will use PDBFixer. \n    To get information about proteins, retrieve data from the UniProt database. \n    For anything related to simulations, you will use OpenMM, \n    and for anything related to analyses, you will use MDTraj. \n\n    You can only respond with a single complete\n    \'Thought, Action, Action Input\' format\n    OR a single \'Final Answer\' format.\n\n    Complete format:\n    Thought: (reflect on your progress 

In [7]:
now = datetime.datetime.now()
date = now.strftime("%Y-%m-%d")
print("date:",date)
time = now.strftime("%H:%M:%S")
print("time:",time)

date: 2024-10-17
time: 23:51:11


In [1]:
# Agent had trouble using libraries so it gave the script instead
# Testing generated code

import requests
from pdbfixer import PDBFixer
from openmm.app import PDBFile

# Download the PDB file for protein 1A3N
def download_pdb(pdb_id):
    url = f'https://files.rcsb.org/download/{pdb_id}.pdb'
    response = requests.get(url)
    if response.status_code == 200:
        with open(f'{pdb_id}.pdb', 'w') as file:
            file.write(response.text)
    else:
        raise Exception('Failed to download PDB file')

# Clean the PDB file using PDBFixer
def clean_pdb(pdb_id):
    fixer = PDBFixer(filename=f'{pdb_id}.pdb')
    fixer.findMissingResidues()
    fixer.findNonstandardResidues()
    fixer.replaceNonstandardResidues()
    fixer.removeHeterogens(keepWater=False)
    fixer.findMissingAtoms()
    fixer.addMissingAtoms()
    fixer.addMissingHydrogens(7.0)
    PDBFile.writeFile(fixer.topology, fixer.positions, open(f'{pdb_id}_clean.pdb', 'w'))

# Execute the functions
download_pdb('1A3N')
clean_pdb('1A3N')
print('Protein 1A3N downloaded and cleaned successfully.')

Protein 1A3N downloaded and cleaned successfully.


In [3]:
# checking files AFTER testing the generated code (there were none earlier)
!ls

1A3N_clean.pdb	1A3N.pdb  exp_21.ipynb


In [4]:
# check the changes in cleaning
import mdtraj as md
path_before = "1A3N.pdb"
path_after = "1A3N_clean.pdb"

raw_file = md.load(path_before)
clean_file = md.load(path_after)

residues_before = set([str(res)[:3] for res in raw_file.top.residues])
residues_after = set([str(res)[:3] for res in clean_file.top.residues])
print("Residues before:",residues_before)
print("Residues after:",residues_after)
print("Residues removed:",residues_before - residues_after)

Residues before: {'ALA', 'CYS', 'ARG', 'HOH', 'TYR', 'GLY', 'TRP', 'LEU', 'ASN', 'GLN', 'PHE', 'THR', 'SER', 'LYS', 'GLU', 'ASP', 'HIS', 'HEM', 'VAL', 'PRO', 'MET'}
Residues after: {'ALA', 'CYS', 'ARG', 'TYR', 'GLY', 'TRP', 'LEU', 'ASN', 'GLN', 'PHE', 'THR', 'SER', 'LYS', 'GLU', 'ASP', 'HIS', 'VAL', 'PRO', 'MET'}
Residues removed: {'HOH', 'HEM'}
